In [1]:
from selenium import webdriver
import time
from tqdm import tqdm_notebook
import pandas as pd
import glob as gb

In [2]:
# generate Webdriver object - Chrome version 79.0.3945.130 (64 bit)
driver = webdriver.Chrome('driver/chromedriver.exe')
driver.get('https://nid.naver.com/nidlogin.login')

## <<< Login manually at this step >>>

## (manually executed below cells, according to menus) - to separate directories

In [116]:
driver.get('https://cafe.naver.com/mpihb')  # //cafe.naver.com/MyCafeIntro.nhn?clubid=23064234
driver.implicitly_wait(1)

menu_list = driver.find_elements_by_xpath('//*[@id="group372"]/li')  # list with 3 items  # manually changed line
menu_list[2].click()  # manually changed line
driver.implicitly_wait(5)

In [117]:
driver.switch_to.frame('cafe_main')
driver.implicitly_wait(5)
driver.find_element_by_xpath('//*[@id="listSizeSelectDiv"]/a').click()
driver.find_element_by_xpath('//*[@id="listSizeSelectDiv"]/ul/li[7]/a').click()
time.sleep(1)

In [118]:
page_num = len(driver.find_elements_by_xpath('//*[@id="main-area"]/div[6]/a'))
for page_idx in range(page_num):
    
    page_list = driver.find_elements_by_xpath('//*[@id="main-area"]/div[6]/a')
    page_list[page_idx].click()
    time.sleep(1)

    article_num = len(driver.find_elements_by_xpath('//*[@id="main-area"]/div[4]/table/tbody/tr'))
    print(f'Start crawling {page_idx+1} page, {article_num} items')
    for idx in tqdm_notebook(range(article_num)):
        article_list = driver.find_elements_by_xpath('//*[@id="main-area"]/div[4]/table/tbody/tr')
        time.sleep(1)
        tmp_article = article_list[idx].find_element_by_tag_name('td').find_elements_by_tag_name('div')[2].find_element_by_tag_name('a')
        tmp_article.click()
        time.sleep(1)

        try:
            driver.find_element_by_xpath('//a[@class="atch_view m-tcol-c"]').click()
            btn_list = driver.find_elements_by_xpath('//div[@class="download_opt"]/a[1]')
            for btn_idx in range(len(btn_list)):
                driver.find_elements_by_xpath('//div[@class="download_opt"]/a')[btn_idx].click()
                time.sleep(3)
        except Exception as e:
            print('>>> Error!!!!! :\n',e)            

        driver.execute_script("window.history.go(-1)")
        time.sleep(3)

print('>>> crawling successfully finishied')

Start crawling 1 page, 45 items


C:\Users\student\Anaconda3\envs\homeET\lib\site-packages\ipykernel_launcher.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # Remove the CWD from sys.path while we load stuff.



>>> crawling successfully finishied


## Another Version (moving to next article directly)

- not good for Debugging

In [102]:
driver.get('https://cafe.naver.com/mpihb')  # //cafe.naver.com/MyCafeIntro.nhn?clubid=23064234
driver.implicitly_wait(1)

menu_list = driver.find_elements_by_xpath('//*[@id="group372"]/li')  # list with 3 items  # manually changed line
menu_list[2].click()  # manually changed line
driver.implicitly_wait(5)

driver.switch_to.frame('cafe_main')
driver.implicitly_wait(5)

driver.find_element_by_xpath('//*[@id="main-area"]/div[4]/table/tbody/tr[1]/td[1]/div[2]/div/a').click()
driver.implicitly_wait(5)

article_cnt = 0
while True:
    article_cnt += 1
    
#     if article_cnt not in (20-1, 28-1, 32-1, 35-1, 39-1, 40-1, 45-1):
#         continue
    
    try:
        driver.find_element_by_xpath('//a[@class="atch_view m-tcol-c"]').click()
        btn_list = driver.find_elements_by_xpath('//div[@class="download_opt"]/a[1]')
        for btn_idx in range(len(btn_list)):
            driver.find_elements_by_xpath('//div[@class="download_opt"]/a')[btn_idx].click()
            time.sleep(3)
    except Exception as e:
        print(f'>>> Error in article # {article_cnt}!!!!! :\n',e)
    
    
    try:
        next_btn = driver.find_element_by_xpath('//*[@id="main-area"]/div[1]/div[1]/div[2]/p/a')
        next_btn.click()
        driver.implicitly_wait(5)
    except:
        try:
            time.sleep(1)
            next_btn = driver.find_element_by_xpath('//*[@id="main-area"]/div[1]/div[1]/div[2]/p/a')
            next_btn.click()
            driver.implicitly_wait(5)
        except Exception as e:
            print(f'>>> crawling may be successfully finishied at article # {article_cnt}')
            break
    
    if article_cnt%20 == 0:
        print(f'>>> successfully crawled article # {article_cnt}')    
    
#     if article_cnt > 50:
#         break
    
    time.sleep(1)
    pass

KeyboardInterrupt: 